In [1]:
import pandas as pd
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
import dash
from dash import Dash, html, dcc
# import dash_core_components as dcc 
# import dash_html_components as html
from dash.dependencies import Input, Output, State
import datetime

# from visualize import Visualize2
import random
# from google.cloud import bigquery
# from google.oauth2 import service_account
import pandas as pd
import numpy as np
import json

In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(
'./chain-coders-5d47d58b2c33.json')

project_id = 'chain-coders'
client = bigquery.Client(credentials= credentials,project=project_id)
# query = "SELECT * FROM `redfin.redfin-data-state`"
# df = client.query(query).to_dataframe()

In [3]:
app = dash.Dash(__name__)

In [4]:
# -- Import and clean data (importing csv into pandas)
HPI_state = pd.read_csv('./data/HPI_AT_state.csv', header=None)
HPI_state.columns = ['state_abbr', 'year', 'quarter', 'HPI_base_1980']
HPI_state['year_quarter'] = HPI_state.year.astype(str)[:]+'-Q'+HPI_state.quarter.astype(str)[:]
HPI_state.year_quarter = HPI_state.year_quarter.str.replace(r'(\d{4})_(\d)', r'\1-Q\2')
HPI_state.year_quarter = pd.to_datetime(HPI_state.year_quarter).dt.date
df = HPI_state.reindex(['state_abbr', 'year_quarter', 'HPI_base_1980'], axis=1)

#property types for drop down
property_set = ['']

#metric types for dropdown
metric_set = ['HPI_base_1980']

#dates for slider
sorted_dates = np.sort(df['year_quarter'].unique())
sorted_dates = ['{}'.format(i) for i in sorted_dates] #convert each date to string within a list
counter = 0
year = 1975

slider_labels = {}
#labels for slider
for index, date in enumerate(sorted_dates):
       if counter == index:
              slider_labels['{}'.format(index)] = year
              year += 1
              counter += 4
       else: 
              slider_labels['{}'.format(index)] = ''

# labels = {index: date for index, date in enumerate(sorted_dates)} #labels of dates for slider object

#color-scale
color_scale = [[0.0, '#edf8fb'],[0.2, '#ccece6'],[0.4, '#99d8c9'],
       [0.6, '#66c2a4'],[0.8, '#2ca25f'],[1.0, '#006d2c']] # purples


/var/folders/q5/w70sk57n1wg65pgbm0rks01h0000gp/T/ipykernel_35098/3411183983.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  HPI_state.year_quarter = HPI_state.year_quarter.str.replace(r'(\d{4})_(\d)', r'\1-Q\2')


In [ ]:
print(property_set)

In [ ]:
print(sorted_dates)
print(len(sorted_dates))

In [ ]:
# line = go.Figure(data=[go.Scatter(x=[1, 2, 3], y=[4, 1, 2])])

In [5]:
%store -r state_codes

In [6]:
state_codes['MT'] = '30'
state_codes['ND'] = '38'
state_codes['WY'] = '56'

In [15]:
import pickle as pkl

In [ ]:
with open(r"state_codes.pickle", "wb") as output_file:
    pkl.dump(d, output_file)

In [7]:
df['state_code'] = df.state_abbr.apply(state_codes.get)

In [5]:
df2 = pd.read_csv('pred_HPI_state.csv')

In [6]:
df2.year_quarter = pd.to_datetime(df2.year_quarter)

In [7]:
df2.rename(columns={'HPI_base_1980': 'predicted', 'test': 'actual'}, inplace=True)

In [ ]:
# temp = df2[(df2['property'] == 'All Residential') & (df2['state'] == 'MN')]
# print(temp)

In [12]:
sorted_dates

['1975-01-01',
 '1975-04-01',
 '1975-07-01',
 '1975-10-01',
 '1976-01-01',
 '1976-04-01',
 '1976-07-01',
 '1976-10-01',
 '1977-01-01',
 '1977-04-01',
 '1977-07-01',
 '1977-10-01',
 '1978-01-01',
 '1978-04-01',
 '1978-07-01',
 '1978-10-01',
 '1979-01-01',
 '1979-04-01',
 '1979-07-01',
 '1979-10-01',
 '1980-01-01',
 '1980-04-01',
 '1980-07-01',
 '1980-10-01',
 '1981-01-01',
 '1981-04-01',
 '1981-07-01',
 '1981-10-01',
 '1982-01-01',
 '1982-04-01',
 '1982-07-01',
 '1982-10-01',
 '1983-01-01',
 '1983-04-01',
 '1983-07-01',
 '1983-10-01',
 '1984-01-01',
 '1984-04-01',
 '1984-07-01',
 '1984-10-01',
 '1985-01-01',
 '1985-04-01',
 '1985-07-01',
 '1985-10-01',
 '1986-01-01',
 '1986-04-01',
 '1986-07-01',
 '1986-10-01',
 '1987-01-01',
 '1987-04-01',
 '1987-07-01',
 '1987-10-01',
 '1988-01-01',
 '1988-04-01',
 '1988-07-01',
 '1988-10-01',
 '1989-01-01',
 '1989-04-01',
 '1989-07-01',
 '1989-10-01',
 '1990-01-01',
 '1990-04-01',
 '1990-07-01',
 '1990-10-01',
 '1991-01-01',
 '1991-04-01',
 '1991-07-

In [8]:
# ------------------------------------------------------------------------------
# App layout
#what goes inside dash layout are the dash components and any html we need
app.layout = html.Div([

    html.H1("HPI Choropleth Data", style={'text-align': 'center'}), #html Title

#     html.Div(id='output_container', children=["Metric"], style = {'font-weight': 'bold'}),

    #components, e.g., drop downs, sliders
    #first drop down by metric
    dcc.Dropdown(id="metric",
                 placeholder = "Select Metric",
                 options=metric_set,
                 value=metric_set[0],
                 style={'width': "60%",}
                 ),

    html.Br(), #space between dropdowns

#     html.Div(id='output_container', style = {'font-weight': 'bold'}, 
#     children = ["Property Type"]
#     ),

    #second drop down for property type
    dcc.Dropdown(id="property_type",
                 placeholder = "Select Property Type",
                 options=property_set,
                 value=property_set[0],
                 style={'width': "60%"},
                 ),

    
    html.Br(), #space
    
    #create div element, e.g., text to display dropdown selection
    html.Div(id='output_container', children=[]),

    html.Br(), #space

    #graph object, e.g., choropleth
    dcc.Graph(id='choropleth', figure={}),

    

    #slider object
    dcc.Slider(0, len(sorted_dates), step = None, #min value of 0 and max of number of unique dates
               value=0, #where the slider starts
               marks = slider_labels,
               tooltip={"placement": "bottom", "always_visible": True}, #place slider at bottom of graph
               id='date_chosen'),

    #add line chart
    dcc.Graph(id = "line_chart", figure = {})

])

In [12]:
df[df.year_quarter==pd.to_datetime(sorted_dates[0])]

/Users/enrico.eduardo/opt/miniconda3/envs/plutho/lib/python3.10/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


,state_abbr,year_quarter,HPI_base_1980,state_code
0,AK,1975-01-01,62.10,02
190,AL,1975-01-01,75.03,01
380,AR,1975-01-01,66.69,05
570,AZ,1975-01-01,60.50,04
760,CA,1975-01-01,41.71,06
950,CO,1975-01-01,54.19,08
1140,CT,1975-01-01,62.06,09
1330,DC,1975-01-01,46.56,11
1520,DE,1975-01-01,77.16,10
1710,FL,1975-01-01,65.89,12


In [9]:
@app.callback(
    Output(component_id='output_container', component_property='children'),
    Output(component_id='choropleth', component_property='figure'),
    Output(component_id = 'line_chart', component_property = 'figure'),
    Input(component_id='property_type', component_property='value'),
    Input(component_id = 'metric', component_property = 'value'),
    Input(component_id='date_chosen', component_property='value'),
    Input(component_id = 'choropleth', component_property = 'hoverData'))


def update_graph(property_type, metric, date_chosen, hover):
    print(property_type)
    print(type(property_type))
    # print(hover)
    # print(type(hover))

    container = "Date Selected: {}".format(sorted_dates[date_chosen])

    dff = df.copy()
    dff = dff[dff['year_quarter'] == pd.to_datetime(sorted_dates[date_chosen])]

    # Plotly Express
    fig = px.choropleth(
        data_frame=dff,
        locationmode='USA-states',
        locations='state_abbr',
        scope="usa",
        color='{}'.format(metric),
        hover_data=['state_abbr', '{}'.format(metric)],
        color_continuous_scale= color_scale
    )

    if type(hover) == dict:
        hover_state = hover['points'][0]['customdata'][0]
        hover_df = df2[df2['state'] == hover_state]
        # hover_df = hover_df[['year', '{}'.format(metric)]]
        # hover_x = hover_df['year'].values.tolist()
        # hover_y = hover_df['{}'.format(metric)].values.tolist()
        # hover_y = [int(year) for year in hover_y]
        line_data = px.line(hover_df, x = "year_quarter", y = ['predicted', 'actual'], title = '{fmetric} for {fstate}'.format(fmetric = metric, fstate = hover_state))
        # line_data = go.Figure(data=[go.Scatter(x=hover_x, y=hover_y)])
        result = line_data
    else:
        result = []

    return container, fig, result

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [04/Dec/2022 00:05:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2022 00:05:49] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2022 00:05:49] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2022 00:05:49] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Dec/2022 00:05:49] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Dec/2022 00:05:49] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Dec/2022 00:05:49] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
/Users/enrico.eduardo/opt/miniconda3/envs/plutho/lib/python3.10/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts 


<class 'str'>


127.0.0.1 - - [04/Dec/2022 00:05:50] "POST /_dash-update-component HTTP/1.1" 200 -
/Users/enrico.eduardo/opt/miniconda3/envs/plutho/lib/python3.10/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

/Users/enrico.eduardo/opt/miniconda3/envs/plutho/lib/python3.10/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

127.0.0.1 - - [04/Dec/2022 00:05:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2022 00:05:53] "POST /_dash-update-component HTTP/1.1" 200 -



<class 'str'>

<class 'str'>


...


<class 'str'>

<class 'str'>


/Users/enrico.eduardo/opt/miniconda3/envs/plutho/lib/python3.10/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

127.0.0.1 - - [04/Dec/2022 00:09:02] "POST /_dash-update-component HTTP/1.1" 200 -
/Users/enrico.eduardo/opt/miniconda3/envs/plutho/lib/python3.10/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

127.0.0.1 - - [04/Dec/2022 00:09:02] "POST /_dash-update-component HTTP/1.1" 200 -



<class 'str'>

<class 'str'>


/Users/enrico.eduardo/opt/miniconda3/envs/plutho/lib/python3.10/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

127.0.0.1 - - [04/Dec/2022 00:09:02] "POST /_dash-update-component HTTP/1.1" 200 -



<class 'str'>
